In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:
BASE_URL = ""

In [ ]:
def land_page(url):
    driver.get(url)

In [ ]:
def close_dialogue():
    driver.find_element(By.CSS_SELECTOR, 'div[aria-label="Close"]').click()

In [ ]:
def login_dialogue(username, password):
    email_feild = driver.find_element(By.ID, ':r1k:')
    password_feild = driver.find_element(By.CSS_SELECTOR, 'label[aria-label="Password"]')
    email_feild.send_keys(username)
    password_feild.send_keys(password)
    login_btn = driver.find_element(By.CSS_SELECTOR, 'div[aria-label="Accessible login button"]')
    login_btn.click()

In [ ]:
def click_comments(c_type):
    comment_section = driver.find_element(By.CSS_SELECTOR, '.x1jx94hy.x78zum5.xdt5ytf.x2lah0s.xw2csxc.x1odjw0f.xmd6oqt.x13o0s5z')
    caption = comment_section.find_element(By.CSS_SELECTOR, 'div[dir="auto"]')
    most_rel_btn = comment_section.find_element(By.CSS_SELECTOR, 'span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.x1s688f.xi81zsa')
    most_rel_btn.click()
    comment_type_box = driver.find_element(By.CSS_SELECTOR, '[role="menu"].x1n2onr6.xcxhlts.x1fayt1i')
    comment_type = comment_type_box.find_elements(By.TAG_NAME, 'span')
    comment_type[c_type].click()
    time.sleep(5)
    uls = comment_section.find_element(By.TAG_NAME, 'ul')
    lis = uls.find_elements(By.TAG_NAME, 'li')
    return lis


In [ ]:
def get_data(lis):
    data = []
    count = 0
    for i in range(len(lis)):
        data.append(lis[i].text)
    return data

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)
land_page(BASE_URL)

In [ ]:
lis = click_comments(4)
data = get_data(lis)
filteredData = []
for i in data:
    if len(i) > 10:
        j = i.split('\n')
        name = j[0]
        comment = j[1]
        filteredData.append([name, comment])
df = pd.DataFrame(filteredData, columns=['name', 'comment'])
df

In [ ]:
df.to_csv('comments_sample.csv')

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sia = SentimentIntensityAnalyzer()

# Function to determine sentiment label
def get_sentiment_label(sentiment_score):
    if sentiment_score > 0:
        return 1  # Positive
    elif sentiment_score < 0:
        return -1  # Negative
    else:
        return 0  # Neutral

In [ ]:
df= pd.read_csv('comments_sample.csv')

In [ ]:
# Apply sentiment analysis and add sentiment column to DataFrame
df['sentiment_scores'] = df['comment'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['sentiment'] = df['sentiment_scores'].apply(get_sentiment_label)

In [ ]:
df

In [ ]:
print(len(df))
print(df['sentiment'].value_counts())

In [ ]:
total_positive = (df['sentiment'] == 1).sum()
total_negative = (df['sentiment'] == -1).sum()
total_neutral = (df['sentiment'] == 0).sum()

# Calculate the total number of comments
total_comments = total_positive + total_negative + total_neutral

# Calculate the proportion of each sentiment category
proportion_positive = total_positive / total_comments
proportion_negative = total_negative / total_comments
proportion_neutral = total_neutral / total_comments

# Determine the overall sentiment based on the majority proportion
if proportion_positive > proportion_negative:
    overall_sentiment = "Positive"
elif proportion_positive < proportion_negative:
    overall_sentiment = "Negative"
else:
    overall_sentiment = "Neutral"

# Print or display the results
print(f"Total Comments: {total_comments}")
print(f"Proportion Positive: {proportion_positive:.2%}")
print(f"Proportion Negative: {proportion_negative:.2%}")
print(f"Proportion Neutral: {proportion_neutral:.2%}")
print(f"Overall Sentiment: {overall_sentiment}")